In [1]:
import pandas as pd
import sqlite3

## Создаем подключение к базе данных

In [2]:
connection = sqlite3.connect('data/checking-logs.sqlite')

## Получить схему таблицы test

In [3]:
schema_query = 'PRAGMA table_info(test);'
schema = pd.io.sql.read_sql(schema_query, connection)
print('Схема таблицы test')
schema

Схема таблицы test


,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## Получить только первые 10 строк таблицы test

In [4]:
sample_query = 'SELECT * FROM test LIMIT 10'
sample = pd.io.sql.read_sql(sample_query, connection)
print('Первые 10 строк таблицы test')
sample

Первые 10 строк таблицы test


,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18,2020-04-26 21:53:59
1,user_1,laba04s,2020-04-26 17:12:11,2020-04-26 21:53:59
2,user_1,laba05,2020-05-02 19:15:18,2020-04-26 21:53:59
3,user_1,laba06,2020-05-17 16:26:35,2020-04-26 21:53:59
4,user_1,laba06s,2020-05-20 12:23:37,2020-04-26 21:53:59
5,user_1,project1,2020-05-14 20:56:08,2020-04-26 21:53:59
6,user_10,laba04,2020-04-25 08:24:52,2020-04-18 12:19:50
7,user_10,laba04s,2020-04-25 08:37:54,2020-04-18 12:19:50
8,user_10,laba05,2020-05-01 19:27:26,2020-04-18 12:19:50
9,user_10,laba06,2020-05-19 11:39:28,2020-04-18 12:19:50


## Найти среди всех пользователей минимальное значение дельты между первым коммитом пользователя и крайним сроком соответствующей лабораторной работы, используя только один запрос
- разница должна отображаться в часах
- не принимайте во внимание лабораторную работу «project1», у нее более длительные сроки выполнения, и она будет выделяться
- значение должно быть сохранено в фрейме данных df_min с соответствующим uid

In [5]:
min_query = '''
SELECT
    t.uid,
    MIN((JULIANDAY(DATETIME(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24) AS min_diff
FROM test t
JOIN deadlines d ON t.labname = d.labs
WHERE t.labname != 'project1'
GROUP BY t.uid
'''
df_min = pd.io.sql.read_sql(min_query, connection)
print('Минимальное значение дельты')
df_min

Минимальное значение дельты


,uid,min_diff
0,user_1,6.796667
1,user_10,39.368056
2,user_14,84.448611
3,user_17,34.643056
4,user_18,3.934167
5,user_19,32.729444
6,user_21,33.905278
7,user_25,2.867500
8,user_28,8.104167
9,user_3,60.511667


## Сделайте то же самое, но для максимума, используя только один запрос, имя dataframe будет df_max

In [6]:
max_query = '''
SELECT
    t.uid,
    MAX((JULIANDAY(DATETIME(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24) AS max_diff
FROM test t
JOIN deadlines d ON t.labname = d.labs
WHERE t.labname != 'project1'
GROUP BY t.uid
'''
df_max = pd.io.sql.read_sql(max_query, connection)
print('Максимальное значение дельты')
df_max

Максимальное значение дельты


,uid,max_diff
0,user_1,175.556667
1,user_10,132.341944
2,user_14,200.766389
3,user_17,81.591667
4,user_18,10.973611
5,user_19,148.916111
6,user_21,126.199722
7,user_25,150.870000
8,user_28,174.853056
9,user_3,182.055278


## Сделайте то же самое, но для среднего значения, используя только один запрос, на этот раз ваш фрейм данных не должен включать столбец uid, а имя фрейма данных - df_avg

In [7]:
avg_query = '''
SELECT
    AVG((JULIANDAY(DATETIME(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24) AS avg_diff
FROM test t
JOIN deadlines d ON t.labname = d.labs
WHERE t.labname != 'project1'
'''
df_avg = pd.io.sql.read_sql(avg_query, connection)
print('Максимальное значение дельты')
df_avg

Максимальное значение дельты


,avg_diff
0,89.687841


## Рассчитать коэффициент корреляции между количеством просмотров страниц и разницей
- Используя только один запрос, создайте таблицу со столбцами: uid, avg_diff, pageviews
- uid — это uid, которые существуют в тесте
- avg_diff — это средняя разница между первым коммитом и крайним сроком выполнения лабораторных работ на пользователя.
- pageviews — это количество посещений новостной ленты на одного пользователя
- не принимайте во внимание лабораторную работу «project1»
- сохраните его в dataframe views_diff

In [8]:
views_diff_query = '''
SELECT
    t.uid,
    AVG((JULIANDAY(DATETIME(d.deadlines, 'unixepoch')) - JULIANDAY(t.first_commit_ts)) * 24) AS avg_diff,
    COUNT(t.uid) as pageviews
FROM test t
JOIN deadlines d ON t.labname = d.labs
WHERE t.labname != 'project1'
GROUP BY t.uid
'''
views_diff = pd.io.sql.read_sql(views_diff_query, connection)
print('Таблица views_diff')
views_diff

Таблица views_diff


,uid,avg_diff,pageviews
0,user_1,65.119778,5
1,user_10,75.242444,5
2,user_14,159.568796,3
3,user_17,62.207667,5
4,user_18,6.368148,3
5,user_19,99.440417,4
6,user_21,96.111181,4
7,user_25,93.474944,5
8,user_28,86.793833,5
9,user_3,105.738222,5


используйте метод Pandas corr() для расчета коэффициента корреляции между количеством просмотров страниц и разницей

In [9]:
correlation = views_diff['avg_diff'].corr(views_diff['pageviews'])
print(correlation)

-0.11837432248290769


## Закрыть соединение

In [10]:
connection.close()